In [14]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [15]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'wall_thickness_metric'

In [16]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [17]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='WallThickness'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

1032


In [18]:
# print(len(df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]))
# df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, WallThickness]
Index: []

In [46]:
df

buckets external_id  \
0     Compressors Accessories & Replacement Parts      08U45T   
1     Compressors Accessories & Replacement Parts      10U50T   
2     Compressors Accessories & Replacement Parts      12U52T   
3     Compressors Accessories & Replacement Parts      12U55T   
4     Compressors Accessories & Replacement Parts    GT470808   
...                                           ...         ...   
1027  Compressors Accessories & Replacement Parts       51106   
1028  Compressors Accessories & Replacement Parts       51036   
1029  Compressors Accessories & Replacement Parts       51035   
1030  Compressors Accessories & Replacement Parts       51104   
1031  Compressors Accessories & Replacement Parts       51107   

                                                   product_name  \
0                     1/2 x 15mm Metric Standpipe Straight Tube   
1                     5/8 x 20mm Metric Standpipe Straight Tube   
2                     3/4 x 22mm Metric Standpipe Straight Tube   
3                     3/4 x 25mm Metric Standpipe Straight Tube   
4     1/2 x 1/2 (30mm) Split Flange (Code 61) Straight Hose End   
...                                                         ...   
1027                              #106 Viton O-Ring (3/32 Wall)   
1028                              #036 Viton O-Ring (1/16 Wall)   
1029                              #035 Viton O-Ring (1/16 Wall)   
1030                              #104 Viton O-Ring (3/32 Wall)   
1031                              #107 Viton O-Ring (3/32 Wall)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   FOR USE WITH HOSE: H017, H039, H104, H115, H145, H245, H324, H425, K245, K425. TYPICAL APPLICATION: General purpose low, medium and high pressure hydraulics. PRESSURE: Determined by maximum working pressure for hose. MATERIAL: Carbon steel - AISI/SAE 12L14. PLATING: Zinc Plated. 'U' SERIES ADVANTAGES: Wide selection of hose and end configurations allowing a diverse number of applications.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [24]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [25]:
# df[curation_col]='n/a'
# matchna=df[['external_id',curation_col]]
# df[0:500]

In [40]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(.{0,20}\d+.{0,3}mm.{0,3}wall.{0,20})|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))

pat=r'(?i)(\d+\.?\d*.?mm(?:\sx?\s|\s?\-\s?)\d*\.?\d*.?m?m?|\d+\.?\d*.?mm)|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

pat=r'(?i)(m)|()'
df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))

custom=df[(df[field].astype(str)!='nan')&((df['mm'].astype(str)!='[]'))]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm',' mm',str(x))).apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r'\.0','',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
matchcustom[curation_col]=matchcustom[curation_col]
matchcustom=matchcustom[matchcustom[curation_col].astype(str)!='']
matchcustom[curation_col].explode().value_counts()

0


Series([], Name: Q:wall_thickness_metric, dtype: int64)

In [41]:
not_match=df[(df[field].astype(str)=='nan')&(df['mm'].astype(str)!='[]')]
print(len(not_match[(not_match['ld'].astype(str)=='[]')|(not_match['name'].astype(str)!='[]')]))
two=not_match[(not_match['ld'].astype(str)=='[]')&(not_match['name'].astype(str)!='[]')]
print(len(two))
two[curation_col]=two['name'].apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))
matchtwo=two[['external_id',curation_col]]
print(len(matchtwo))
two

0
0
0


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, WallThickness, Q:wall_thickness_metric, name, ld, matches, mm]
Index: []

In [42]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [43]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [44]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [45]:
na=df[(df['name'].astype(str)=='[]')&(df[field].astype(str)=='nan')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))

422


In [47]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [18]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [48]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [30]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 